In [1]:
import pandas as pd
import pandas_profiling

data = pd.read_csv("C:\\Users\\KulkovaAV\\Desktop\\Feature engineering\\дз_корректное_\\дз\\data\\aac_shelter_outcomes.csv")

Описание датафрейма
age_upon_outcome - время с последней выгрузки (= datetime - date_of_birth)
animal_id - идентификационный номер животного
animal_type - тип животного
breed - порода (домашняя, короткая шерсть, микс)
color - цвет
date_of_birth - дата рождения
datetime = monthyear - текущая дата?
name - имя
outcome_subtype - партнер, приемный, риск бешенства, страдающий, агрессивный, в питомнике, украден, и т.д.
outcome_type - вернулся к хозяину, умер, перевезен, взят из приюта, усыпление
sex_upon_outcome - пол животного (например, "неповрежденный самец", "стерилизованная самка")

In [2]:
#Информация о датафрейме
data.describe()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
count,78248,78256,78256,78256,78256,78256,78256,78256,54370,35963,78244,78254
unique,46,70855,5,2128,525,5869,64361,64361,14574,19,9,5
top,1 year,A718223,Dog,Domestic Shorthair Mix,Black/White,2014-05-05T00:00:00,2016-04-18T00:00:00,2016-04-18T00:00:00,Bella,Partner,Adoption,Neutered Male
freq,14355,11,44242,23335,8153,112,39,39,344,19660,33112,27784


In [3]:
#Количество пропущенных значений по каждому столбцу
data.isnull().sum()

age_upon_outcome        8
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
datetime                0
monthyear               0
name                23886
outcome_subtype     42293
outcome_type           12
sex_upon_outcome        2
dtype: int64

In [4]:
# Нет ни одной записи, где datetime != monthyear => это два одинаковых столбца. Удалим один из них.
data[data.datetime != data.monthyear].head()
data = data.drop('monthyear', axis = 1)

In [5]:
#Количество пустых записей в столбце name = 23886. Нет смысла заполнять его какими-то записями, 
#т.к. для анализа - имя животного не важно.Следовательно, можем удалить данный столбец.
data = data.drop('name', axis = 1)

In [6]:
# Убрала неизвестные значения в столбце age_upon_outcome.Перевела в формат даты и оставила дни.  
# Т.к. age_upon_outcome = datetime - date_of_birth => можно убрать столбец datetime, к примеру.
from datetime import datetime
data['age_upon_outcome'] = data['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')) - data['date_of_birth'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
data['age_upon_outcome'] = data['age_upon_outcome'].apply(lambda x: x.days)
data = data.drop('datetime', axis = 1)

In [7]:
#Удаляем столбец date_of_birth т.к. для обучения он не нужен.  
data = data.drop(['date_of_birth'], axis = 1)

In [8]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome
0,15,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male
1,366,A666430,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female
2,429,A675708,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male
3,3300,A680386,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male
4,181,A683115,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown


In [9]:
pandas_profiling.ProfileReport(data)

Number of variables,8
Number of observations,78256
Total Missing (%),6.8%
Total size in memory,4.8 MiB
Average record size in memory,64.0 B
Numeric,1
Categorical,7
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [10]:
# Датасет имеем 23 дублирующие строки - удаляем их.
data = data.drop_duplicates()

In [11]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome
0,15,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male
1,366,A666430,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female
2,429,A675708,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male
3,3300,A680386,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male
4,181,A683115,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown


In [12]:
#Чаще всего в столбце "outcome_type" встречается Adoption = 33112. Заменим пропущенные значения на "Adoption"
data.loc[data.outcome_type.isnull() == True, 'outcome_type'] = 'Adoption'

In [41]:
#Чаще всего в столбце "sex_upon_outcome" встречается Neutered Male = 27784. Заменим пропущенные значения на "Neutered Male"
data.loc[data.sex_upon_outcome.isnull() == True, 'sex_upon_outcome'] = 'Neutered Male'

In [42]:
df = data.copy()

In [43]:
#Оставим ту часть dataframe, которой соответствует условию задачи: 'Adoption', 'Transfer'
df = df.loc[(data.outcome_type == 'Adoption') | (data.outcome_type == 'Transfer')]

In [44]:
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome
0,15,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male
1,366,A666430,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female
2,429,A675708,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male
3,3300,A680386,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male
5,126,A664462,Dog,Leonberger Mix,Brown/White,Partner,Transfer,Intact Male


In [45]:
#Количество пропущенных значений по каждому столбцу
df.isnull().sum()

age_upon_outcome        0
animal_id               0
animal_type             0
breed                   0
color                   0
outcome_subtype     27185
outcome_type            0
sex_upon_outcome        0
dtype: int64

In [46]:
pandas_profiling.ProfileReport(df)

Number of variables,9
Number of observations,56604
Total Missing (%),5.3%
Total size in memory,3.9 MiB
Average record size in memory,72.0 B
Numeric,2
Categorical,7
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [47]:
# Т.к. столбец outcome_subtype имеет всего 7 уникальных значений, одно из которых неопределено, 
# а число пропущенных значений = 27185 (не можем удалить 48% записей), 
# то я предлагаю придумать новый признак и подставить вместо nan. Например, "unknown_subtype" 
df.outcome_subtype.unique()

array(['Partner', nan, 'Offsite', 'Foster', 'SCRP', 'Barn', 'Snr'],
      dtype=object)

In [48]:
#Теперь наш датафрейм не имеет пропусков.
df.loc[df.outcome_subtype.isnull() == True, 'outcome_subtype'] = 'Unknown_subtype'

In [49]:
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome
0,15,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male
1,366,A666430,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female
2,429,A675708,Dog,Pit Bull,Blue/White,Unknown_subtype,Adoption,Neutered Male
3,3300,A680386,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male
5,126,A664462,Dog,Leonberger Mix,Brown/White,Partner,Transfer,Intact Male


In [50]:
# Преобразуем категориальные признаки в числовые данные.
animal_type_dict = {'Cat':0,'Dog':1, 'Other':2, 'Bird':3, 'Livestock':4}
sex_upon_outcome_dict = {'Intact Male':0, 'Spayed Female':1, 'Neutered Male':2, 'Intact Female':3, 'Unknown':4}
outcome_subtype_dict = {'Partner':0, 'Unknown_subtype':1, 'Offsite':2, 'Foster':3, 'SCRP':4, 'Barn':5, 'Snr':6}
outcome_type_dict = {'Transfer':0, 'Adoption':1}

df['animal_type_'] = df['animal_type'].map(animal_type_dict)
df['sex_upon_outcome_'] = df['sex_upon_outcome'].map(sex_upon_outcome_dict)
df['outcome_subtype_'] = df['outcome_subtype'].map(outcome_subtype_dict)
df['outcome_type_'] = df['outcome_type'].map(outcome_type_dict)

In [51]:
df = df.drop(['animal_type', 'sex_upon_outcome','outcome_subtype','outcome_type' ], axis = 1)

In [52]:
df.head()

,age_upon_outcome,animal_id,breed,color,animal_type_,sex_upon_outcome_,outcome_subtype_,outcome_type_
0,15,A684346,Domestic Shorthair Mix,Orange Tabby,0,0,0,0
1,366,A666430,Beagle Mix,White/Brown,1,1,0,0
2,429,A675708,Pit Bull,Blue/White,1,2,1,1
3,3300,A680386,Miniature Schnauzer Mix,White,1,2,0,0
5,126,A664462,Leonberger Mix,Brown/White,1,0,0,0


In [53]:
df.describe()

,age_upon_outcome,animal_type_,sex_upon_outcome_,outcome_subtype_,outcome_type_
count,56604.000000,56604.000000,56604.000000,56604.000000,56604.000000
mean,632.784044,0.561232,1.644477,1.081054,0.584959
std,908.070266,0.545199,0.993873,1.235814,0.492734
min,-123.000000,0.000000,0.000000,0.000000,0.000000
25%,78.000000,0.000000,1.000000,0.000000,0.000000
50%,312.000000,1.000000,2.000000,1.000000,1.000000
75%,742.000000,1.000000,2.000000,1.000000,1.000000
max,9137.000000,4.000000,4.000000,6.000000,1.000000


In [54]:
pandas_profiling.ProfileReport(df)

Number of variables,9
Number of observations,56604
Total Missing (%),0.0%
Total size in memory,3.9 MiB
Average record size in memory,72.0 B
Numeric,5
Categorical,3
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [55]:
df.head()

,age_upon_outcome,animal_id,breed,color,animal_type_,sex_upon_outcome_,outcome_subtype_,outcome_type_
0,15,A684346,Domestic Shorthair Mix,Orange Tabby,0,0,0,0
1,366,A666430,Beagle Mix,White/Brown,1,1,0,0
2,429,A675708,Pit Bull,Blue/White,1,2,1,1
3,3300,A680386,Miniature Schnauzer Mix,White,1,2,0,0
5,126,A664462,Leonberger Mix,Brown/White,1,0,0,0


In [56]:
y = df.outcome_type_
X = df[['animal_type_', 'sex_upon_outcome_','outcome_subtype_']]

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [58]:
#Обучение
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
model = RandomForestClassifier(n_estimators = 100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [59]:
#Качество модели
model.score(X_test, y_test)

0.9999116685805141

In [60]:
#Результат сравнения
df_=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df_.head(10)

,Actual,Predicted
13419,1,1
72702,1,1
8993,1,1
60612,1,1
6493,0,0
6150,1,1
76555,0,0
59140,0,0
24680,1,1
77506,1,1
